In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import re
from sklearn.manifold import TSNE
sys.path.append('/home/machao/anaconda2/lib/python2.7/site-packages')
import gensim
from gensim import corpora
from nltk.corpus import stopwords
% matplotlib inline

from gensim.models.word2vec import LineSentence  
from gensim.models import Word2Vec  

In [2]:
data = pd.read_csv('train.csv')
# test = pd.read_csv('test.csv')
stoplist = set(stopwords.words("english"))

In [3]:
def clean_sentence(val):
    val = str(val)
    if not val:
        return ''
    regex = re.compile('([^\s\w]|_)+')
    sentence = regex.sub('', val).lower()
    sentence = sentence.split(" ")
    for word in list(sentence):
        if word in stoplist:
            sentence.remove(word)  
            
    sentence = " ".join(sentence)
    return sentence

def clean_dataframe(data):
#     data = data.dropna(how="any")
    for col in ['question1', 'question2']:
        data[col] = data[col].map(clean_sentence)
    return data


In [4]:
data = clean_dataframe(data)
print data.shape

(404290, 6)


In [5]:
# test = clean_dataframe(test)
# print test.shape

In [6]:
sentences1 = data['question1'].values
sentences2 = data['question2'].values
sentences = np.concatenate((sentences1, sentences2),axis=0)

# sentences1_test = test['question1'].values
# sentences2_test = test['question2'].values
# sentences_test = np.concatenate((sentences1_test, sentences2_test),axis=0)

In [7]:
# sentences_all = np.concatenate((sentences, sentences_test),axis=0)

In [8]:
with open('sentences', 'w') as f:
    for sentence in sentences:
        f.write(str(sentence) + '\n')

In [31]:
texts = [[word for word in str(sentence).lower().split()] for sentence in sentences]

In [33]:
dictionary = corpora.Dictionary(texts)
dictionary.id2token = {value:key for key,value in dictionary.token2id.items()}
dictionary.save('sentences.dict')
# dictionary = corpora.Dictionary.load('sentences.dict')
# print dictionary.token2id

In [34]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('corpus.mm', corpus)

from gensim.models import TfidfModel 
tfidf = TfidfModel(corpus)
tfidf.save('tfidf')

In [13]:
sentences_all = LineSentence('sentences')
word2Vec = Word2Vec(sentences_all, size=128, window=5, min_count=5, workers=8)
word2Vec.save('w2c')

In [39]:
def handel_sentence(word2Vec, tfidf, sentence):
    the_text = [word for word in str(sentence).lower().split()]
    the_corpus = dictionary.doc2bow(the_text)
    weight = { i[0]:i[1] for i in tfidf[the_corpus]}
    res = np.zeros(128)
    for id, _ in the_corpus:
        word = dictionary.id2token[id]
        if word in word2Vec:
            res += weight[id] * word2Vec[word]
    return res   

def transform(data):
    values = np.zeros([data.shape[0], 256])
    for i in data.index:
        v1 = handel_sentence(word2Vec, tfidf, data.loc[i, 'question1'])
        v2 = handel_sentence(word2Vec, tfidf, data.loc[i, 'question2'])
        values[i, :] = np.concatenate((v1, v2))
    return pd.DataFrame(values)
    
train_data = transform(data)
# test_data = transform(test)

In [42]:
train_data

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,0.298713,-0.487257,-0.147068,1.016929,-0.705909,-0.100686,0.129516,-0.303316,-0.518298,0.486690,...,1.375736,0.358729,0.011689,1.758300,-0.005838,-0.160472,0.899045,-0.199773,0.048522,1.505721
1,-0.469292,0.303371,-0.668520,-0.171858,-0.239501,-0.052515,-0.710794,-0.641336,-0.017868,0.193099,...,-0.529680,-0.040180,-0.183577,0.683500,-1.382588,0.231189,-1.430478,-1.009024,0.468944,0.058579
2,0.995534,-0.584196,-0.282547,0.012361,-0.239618,1.583589,0.740704,0.756446,0.898023,0.323763,...,1.876929,-0.813264,1.107512,0.669258,0.120655,0.973647,-0.075865,-0.562470,1.193833,0.239944
3,-0.436320,-0.312318,0.656929,-0.243229,0.490824,-1.038181,0.092425,0.182183,0.303338,-0.941570,...,-0.613904,-0.576674,-0.087805,-0.096381,0.070925,-0.525228,0.379256,0.391419,-0.229037,0.443432
4,-1.052431,0.136104,-0.944685,-1.395176,0.672374,0.223617,-1.416357,-0.574939,1.581200,-0.086265,...,-1.131015,1.028261,0.592324,-0.641588,-0.015540,-0.694198,-2.872577,-2.048356,-0.432282,-1.022243
5,-0.457918,0.263225,-0.254266,0.242833,0.659033,0.066051,-0.949854,-1.042417,0.516999,0.111796,...,-1.240037,-0.324744,0.743326,0.024185,0.211058,-0.682966,-1.344981,0.174282,-0.561412,0.363277
6,-0.435288,-0.185100,0.021666,0.035930,-0.593949,-0.290562,-0.590889,-0.303537,0.247749,0.576572,...,-0.673658,-0.273203,-0.674040,0.170203,-0.940510,1.069331,-1.841067,0.005601,1.471571,0.219536
7,0.109741,-0.099896,0.018114,-0.304194,-0.560276,0.072889,0.206549,-0.181957,-0.111464,-0.143837,...,0.055321,-0.075351,-0.058566,-0.441904,0.631053,-0.024869,-0.030290,-0.451256,0.646637,0.353873
8,-0.701773,-1.933678,0.979404,-0.182407,-0.243038,-0.469539,-0.118236,0.909905,1.032535,1.136361,...,-0.445229,0.589273,0.059608,-0.017667,0.410686,0.504414,-1.542925,-0.592249,1.006758,-0.036689
9,0.756681,0.871010,-0.058241,-0.205418,-0.121125,0.463226,-0.147225,-0.107659,0.482692,-0.406244,...,0.150843,-0.622822,-0.739134,0.900994,-0.315722,1.304264,-0.269673,-0.962833,1.570781,-0.616431


In [ ]:
train_data['label'] = data['is_duplicate']
train_data.to_csv('train_data.csv', index = False)